# 🗺️ 14 — DIM_LOCAL (Gold)

- Lê `silver/2018_anonimizado.xlsx` e `silver/2019_anonimizado.xlsx`
- Constrói dimensão de local com UF e Município
- Preferência: UF/Município do Processo; fallback: Cadastro
- Exporta em `gold/output/dim_local.csv`


## 0) Imports

In [ ]:
import pandas as pd
import numpy as np


## 1) Paths robustos

In [ ]:
from pathlib import Path

# ======================================================
# Paths robustos (funciona no VS Code / Jupyter / OneDrive)
# - encontra a raiz do projeto procurando a pasta 'silver'
# ======================================================
CWD = Path().resolve()

def find_project_root(start: Path) -> Path:
    p = start
    for _ in range(12):
        if (p / "silver").exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    raise FileNotFoundError("Não encontrei a pasta 'silver' subindo a árvore. Rode o notebook dentro do repo.")

PROJECT_ROOT = find_project_root(CWD)
SILVER_DIR = PROJECT_ROOT / "silver"
GOLD_DIR = PROJECT_ROOT / "gold"
OUT_DIR = GOLD_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("📌 CWD:", CWD)
print("📌 PROJECT_ROOT:", PROJECT_ROOT)
print("📌 SILVER_DIR:", SILVER_DIR)
print("📌 OUT_DIR:", OUT_DIR)


## 2) Ler Silver (2018 + 2019)

In [ ]:
import pandas as pd
import numpy as np

INPUT_FILES = [
    SILVER_DIR / "2018_anonimizado.xlsx",
    SILVER_DIR / "2019_anonimizado.xlsx",
]

print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)
print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


## 3) Funções auxiliares

In [ ]:
import pandas as pd
import numpy as np

def norm_missing(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    return x.replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))

def pick_first_existing(candidates, df_):
    return next((c for c in candidates if c in df_.columns), None)


## 4) Construir DIM_LOCAL

In [ ]:
uf_proc = norm_missing(df["UF_PROCESSO"]) if "UF_PROCESSO" in df.columns else pd.Series([pd.NA]*len(df))
uf_cad  = norm_missing(df["UF_CADASTRO"]) if "UF_CADASTRO" in df.columns else pd.Series([pd.NA]*len(df))
mun_proc = norm_missing(df["MUNICIPIO_PROCESSO"]) if "MUNICIPIO_PROCESSO" in df.columns else pd.Series([pd.NA]*len(df))
mun_cad  = norm_missing(df["MUNICIPIO_CADASTRO"]) if "MUNICIPIO_CADASTRO" in df.columns else pd.Series([pd.NA]*len(df))

local = pd.DataFrame({
    "uf": uf_proc.fillna(uf_cad).str.upper(),
    "municipio": mun_proc.fillna(mun_cad).str.upper()
})

local["uf"] = local["uf"].replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})
local["municipio"] = local["municipio"].replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

local["municipio"] = local["municipio"].fillna("NÃO INFORMADO")
local = local.dropna(subset=["uf"])

dim_local = local.drop_duplicates().sort_values(["uf","municipio"]).reset_index(drop=True)
dim_local["id_local"] = dim_local.index + 1
dim_local = dim_local[["id_local","uf","municipio"]]

print("✅ DIM_LOCAL pronta:", dim_local.shape)
dim_local.head(15)


## 5) Exportar

In [ ]:
out_file = OUT_DIR / "dim_local.csv"
dim_local.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)
